# Executing individual simulations

This tutorial illustrates how to use Python APIs for simulation tools to programmatically execute individual simulations. This flexibility functionality can be used for a variety of purposes such as for sampling ensembles of stochastic trajectories, scanning and estimating parameters, and co-simulating mutiple models. For co-simulation, we recommend [Vivarium](https://vivarium-collective.github.io/), a Python framework for multi-algorithmic co-simulation.

Information about the capabilities (supported model formats, modeling frameworks, simulation types, simulation algorithms, observables) of each simulation tool is available from [BioSimulators](https://biosimulators.org). These capabilties are described using several ontologies. Please see the [Programmatically getting information about simulation tools from BioSimulators tutorial](../4.%20Finding%20simulation%20tools/Programmatically%20getting%20information%20about%20simulation%20tools%20from%20BioSimulators.ipynb) for more information about how to use this information to identify and use appropriate simulation tools.

## 1. Executing time course simulations (e.g., continuous and discrete kinetic simulations)

Use [BioSimulators-utils](https://github.com/biosimulators/Biosimulators_utils) to describe the desired simulation. BioSimulators-utils provides several classes for describing simulations which parallel the [Simulation Experiment Description Markup Language (SED-ML)](http://sed-ml.org/).

These classes utilize three ontologies:
* Modeling formats: [SED-ML model language URNs](https://sed-ml.org/urns.html)
* Outputs of implicit simulation variables: [SED-ML symbol URNs](https://sed-ml.org/urns.html)
* Simulation algorithms and their parameters: [Kinetic Simulation Algorithm Ontology (KiSAO)](https://github.com/SED-ML/KiSAO)

In [1]:
from biosimulators_utils.sedml.data_model import Task, Model, ModelLanguage, UniformTimeCourseSimulation, Algorithm, Variable, Symbol

For example, use these classes to describe a 10-time unit simulation of the continuous kinetic ([SBO_0000293](https://www.ebi.ac.uk/ols/ontologies/sbo/terms?iri=http%3A%2F%2Fbiomodels.net%2FSBO%2FSBO_0000293)) [Ciliberto et al. morphogenesis checkpoint model](../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml) using the Systems Biology Markup Language (SBML) format ([EDAM:format_2585](https://www.ebi.ac.uk/ols/ontologies/edam/terms?iri=http%3A%2F%2Fedamontology.org%2Fformat_2585), SED-ML model language URN: `sbml`) and the CVODE algorithm ([KISAO_0000019](https://www.ebi.ac.uk/ols/ontologies/kisao/terms?iri=http%3A%2F%2Fwww.biomodels.net%2Fkisao%2FKISAO%23KISAO_0000019)) with the predicted variables recorded at the initial time point and at 10 additionally uniformly spaced steps from the initial time (0) to the final time (10).

In [2]:
task = Task(
    model=Model(
        source='../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml',
        language=ModelLanguage.SBML.value,        
    ),
    simulation=UniformTimeCourseSimulation(
        initial_time=0.,
        output_start_time=0.,
        output_end_time=10.,
        number_of_steps=10,
        algorithm=Algorithm(
            kisao_id='KISAO_0000019',
        ),
    ),
)

Next, use these classes to describe the outputs (e.g., time, concentrations of species, fluxes/rates/velocities of reactions, sizes of compartments) that should be recorded for the simulation. Outputs for variables explicitly defined in SBML files must be described using their [XML XPaths](https://www.w3schools.com/xml/xpath_syntax.asp) (e.g., `/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Clb"]` for the species with id `Clb`). Outputs for variables not explicitly defined in SBML files (e.g., time) must be described using SED-ML symbol URNs (e.g., `urn:sedml:symbol:time` for time).

In [3]:
sbml_uri = 'http://www.sbml.org/sbml/level2/version4'
variables = [
    Variable(
        id='time',
        symbol=Symbol.time.value,
        task=task,
    ),    
    Variable(
        id='clb',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Clb"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
    Variable(
        id='cln',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Cln"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
    Variable(
        id='sbf',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="SBF"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
    Variable(
        id='sic1',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Sic"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
]

Next, import a simulation tool which can execute time course simulations of models defined in SBML with CVODE such as [tellurium](https://biosimulators.org/simulators/tellurium). [BioSimulators](https://biosimulators.org) provides extensive information about the capabilities of each simulation tool which can be used to identify an appopriate tool for a specific simulation. In addition, [runBioSimulations](https://run.biosimulations.org/) provides wizards that can recommend specific simulation tools for specific projects.

In [4]:
import biosimulators_tellurium

Next, use the `exec_sed_task` method to execute the simulation and record the desired output variables. Each Python API validated by BioSimulators provides the same simulation methods with the same function signatures.

In [5]:
outputs, log = biosimulators_tellurium.exec_sed_task(task, variables)

/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: BioSimulatorsWarning:

Model `None` may be invalid.
  - The model file `../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml` may be invalid.
    - The value of the 'sboTerm' attribute on a <species> is expected to be an SBO identifier (http://www.biomodels.net/SBO/). In SBML Level 2 prior to Version 4 it is expected to refer to a participant physical type (i.e., terms derived from SBO:0000236, "participant physical type"); in Versions 4 and above it is expected to refer to a material entity (i.e., terms derived from SBO:0000240, "material entity").
      Reference: L2V4 Section 5
       SBO term 'SBO:0000014' on the <species> is not in the appropriate branch.
      
    - The value of the 'sboTerm' attribute on a <species> is expected to be an SBO identifier (http://www.biomodels.net/SBO/). In SBML Level 2 prior to Version 4 it is expected to refer to a participant physical type (i.e., te

`exec_sed_task` returns two outputs, the results of the requested variables and a diagnostic log of the execution of the simulation.

`outputs` is a dictionary that maps the id of each requested output variable to a `numpy.ndarray` with its results at each requested output time step. Run the following code to print out the predicted values of the variables.

In [6]:
for variable_id, value in outputs.items():
    print('{}: {}'.format(variable_id.ljust(4, ' '), ' '.join('{:6.3f}'.format(v) for v in value)))

time:  0.000  1.000  2.000  3.000  4.000  5.000  6.000  7.000  8.000  9.000 10.000
clb :  0.185  0.002  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
cln :  0.054  0.052  0.051  0.050  0.049  0.049  0.048  0.048  0.048  0.048  0.048
sbf :  0.124  0.036  0.040  0.041  0.042  0.043  0.045  0.046  0.047  0.048  0.050
sic1:  0.004  0.069  0.211  0.311  0.395  0.472  0.543  0.610  0.673  0.733  0.788


`log.simulator_details` contains diagnostic information, such as the value of each parameter of CVODE that tellurium used. Run the following code to print this information.

In [7]:
import yaml
print(yaml.dump(log.simulator_details))

absolute_tolerance: 1.0e-12
initial_time_step: 0.0
max_output_rows: 100000
maximum_adams_order: 12
maximum_bdf_order: 5
maximum_num_steps: 20000
maximum_time_step: 0.0
method: simulate
minimum_time_step: 0.0
multiple_steps: false
relative_tolerance: 1.0e-06
solver: cvode
stiff: true
variable_step_size: false



## 2. Executing steady-state simulations (e.g., flux balance analysis, analysis of logical regulatory networks)

Similarly, first use BioSimulators' classes to describe the simulation that you would like to execute such as a flux balance analysis ([KISAO_0000437](https://www.ebi.ac.uk/ols/ontologies/kisao/terms?iri=http%3A%2F%2Fwww.biomodels.net%2Fkisao%2FKISAO%23KISAO_0000437)) simulation of a constraint-based ([SBO_0000624](https://www.ebi.ac.uk/ols/ontologies/sbo/terms?iri=http%3A%2F%2Fbiomodels.net%2FSBO%2FSBO_0000624)) [model of the central metabolism of *Escherichia coli*](../_data/Escherichia-coli-core-metabolism.xml) encoded in SBML ([EDAM:format_2585](https://www.ebi.ac.uk/ols/ontologies/edam/terms?iri=http%3A%2F%2Fedamontology.org%2Fformat_2585), SED-ML model language URN: `sbml`).

In [8]:
from biosimulators_utils.sedml.data_model import SteadyStateSimulation

In [9]:
task = Task(
    model=Model(
        source='../_data/Escherichia-coli-core-metabolism.xml',
        language=ModelLanguage.SBML.value,        
    ),
    simulation=SteadyStateSimulation(
        algorithm=Algorithm(
            kisao_id='KISAO_0000437',
        ),
    ),
)

In [10]:
sbml_uri = 'http://www.sbml.org/sbml/level3/version1/core'
sbml_fbc_uri = 'http://www.sbml.org/sbml/level3/version1/fbc/version2'
variables = [
    Variable(
        id='obj',
        target='/sbml:sbml/sbml:model/fbc:listOfObjectives/fbc:objective[@fbc:id="obj"]',
        target_namespaces={'sbml': sbml_uri, 'fbc': sbml_fbc_uri},
        task=task,
    ),
     Variable(
        id='ex_glc',
        target='/sbml:sbml/sbml:model/sbml:listOfReactions/sbml:reaction[@id="R_EX_glc__D_e"]',
        target_namespaces={'sbml': sbml_uri, 'fbc': sbml_fbc_uri},
        task=task,
    ),    
]

Next, import a capable simulation tool such as [COBRApy](https://biosimulators.org/simulators/cobrapy).

In [11]:
import biosimulators_cobrapy

Next, use COBRApy to execute the simulation.

In [12]:
outputs, log = biosimulators_cobrapy.exec_sed_task(task, variables)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


Next, print the predicted value of the objective and glucose exchange flux.

In [13]:
for variable_id, value in outputs.items():
    print('{0:}: {1:7.3f}'.format(variable_id.ljust(6, ' '), value))

obj   :   0.874
ex_glc: -10.000


## 3. Simulating variants of models

BioSimulators APIs can modify models and simulate modified models. `exec_sed_task` methods can change the attributes of components of models, such as the values of parameters and initial conditions. In addition to changing the attributes of components of models, `exec_sed_doc` and `exec_sedml_docs_in_combine_archive` methods can also change the structures of models, such as to add, delete, and replace variables.

BioSimulators-utils provides several classes for describing changes to models which parallel the `changeAttribute`, `computeChange`, `setValue`, `addXML`, `removeXML`, and `changeXML` classes of SED-ML. Changes to models encoded in SBML, must be described using XML XPaths for the XML element that corresponds to each model component/attribute.

The following code illustrates how to simulate a variant of the Ciliberto model with the $k_{swe}^{''}$ (`kswe_prime`) parameter increased by 5%.

In [14]:
from biosimulators_utils.sedml.data_model import ModelAttributeChange

In [15]:
task = Task(
    model=Model(
        source='../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml',
        language=ModelLanguage.SBML.value,        
    ),
    simulation=UniformTimeCourseSimulation(
        initial_time=0.,
        output_start_time=0.,
        output_end_time=10.,
        number_of_steps=10,
        algorithm=Algorithm(
            kisao_id='KISAO_0000019',
        ),
    ),
)

sbml_uri = 'http://www.sbml.org/sbml/level2/version4'
variables = [
    Variable(
        id='time',
        symbol=Symbol.time.value,
        task=task,
    ),    
    Variable(
        id='clb',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Clb"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
    Variable(
        id='cln',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Cln"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
    Variable(
        id='sbf',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="SBF"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
    Variable(
        id='sic1',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Sic"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
]

Simulate the original model

In [16]:
outputs, _ = biosimulators_tellurium.exec_sed_task(task, variables)

/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: BioSimulatorsWarning:

Model `None` may be invalid.
  - The model file `../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml` may be invalid.
    - The value of the 'sboTerm' attribute on a <species> is expected to be an SBO identifier (http://www.biomodels.net/SBO/). In SBML Level 2 prior to Version 4 it is expected to refer to a participant physical type (i.e., terms derived from SBO:0000236, "participant physical type"); in Versions 4 and above it is expected to refer to a material entity (i.e., terms derived from SBO:0000240, "material entity").
      Reference: L2V4 Section 5
       SBO term 'SBO:0000014' on the <species> is not in the appropriate branch.
      
    - The value of the 'sboTerm' attribute on a <species> is expected to be an SBO identifier (http://www.biomodels.net/SBO/). In SBML Level 2 prior to Version 4 it is expected to refer to a participant physical type (i.e., te

Modify the model and re-simulate it

In [17]:
task.model.changes.append(
    ModelAttributeChange(
        target='/sbml:sbml/sbml:model/sbml:listOfParameters/sbml:parameter[@id="kswe_prime"]/@value',
        target_namespaces={'sbml': sbml_uri},
        new_value=2.1,
    )
)
outputs, _ = biosimulators_tellurium.exec_sed_task(task, variables)

## 4. Setting the parameters of simulation algorithms to fine-tune their behavior

Most simulation algorithms expose one or more parameters for fine-tuning their behavior. Detailed information about the parameters supported by each simulation tool and algorithm is available from [BioSimulators](https://biosimulators.org). This includes information about the data type and default value of each parameter.

The following code illustrates how to change the values of the absolute ([KISAO_0000211](https://www.ebi.ac.uk/ols/ontologies/kisao/terms?iri=http%3A%2F%2Fwww.biomodels.net%2Fkisao%2FKISAO%23KISAO_0000211)) and relative ([KISAO_0000209](https://www.ebi.ac.uk/ols/ontologies/kisao/terms?iri=http%3A%2F%2Fwww.biomodels.net%2Fkisao%2FKISAO%23KISAO_0000209)) tolerances of CVODE.

In [18]:
from biosimulators_utils.sedml.data_model import AlgorithmParameterChange

In [19]:
task = Task(
    model=Model(
        source='../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml',
        language=ModelLanguage.SBML.value,        
    ),
    simulation=UniformTimeCourseSimulation(
        initial_time=0.,
        output_start_time=0.,
        output_end_time=10.,
        number_of_steps=10,
        algorithm=Algorithm(
            kisao_id='KISAO_0000019',
            changes=[
                AlgorithmParameterChange(
                    kisao_id='KISAO_0000211',
                    new_value=1e-8,
                ),
                AlgorithmParameterChange(
                    kisao_id='KISAO_0000209',
                    new_value=1e-9,
                ),
            ],
        ),
    ),
)
sbml_uri = 'http://www.sbml.org/sbml/level2/version4'
variables = [
    Variable(
        id='time',
        symbol=Symbol.time.value,
        task=task,
    ),    
    Variable(
        id='clb',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Clb"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
]

In [20]:
outputs, log = biosimulators_tellurium.exec_sed_task(task, variables)

In [21]:
print(f'Absolute tolerance: {log.simulator_details["absolute_tolerance"]}')
print(f'Relative tolerance: {log.simulator_details["relative_tolerance"]}')

Absolute tolerance: 1e-08
Relative tolerance: 1e-09


## 5. Executing simulations with alternative algorithms

To reuse simulations across simulation tools, it is often necessary to use different simulation algorithms with each simulation tool. To make this easy, BioSimulators simulation tools can automatically use the [Kinetic Simulation Algorithm Ontology (KiSAO)](https://github.com/SED-ML/KiSAO) to identify and execute similar algorithms.

Use the following code to use CVODE to execute the Ciliberto model with tellurium and also execute this with COPASI, which implements several similar algorithms, including LSODA.

As described in the next section, BioSimulators provide a configuration option to controlling the degree to which simulation tools should automatically substitute algorithms without raising exceptions.

In [22]:
task = Task(
    model=Model(
        source='../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml',
        language=ModelLanguage.SBML.value,        
    ),
    simulation=UniformTimeCourseSimulation(
        initial_time=0.,
        output_start_time=0.,
        output_end_time=10.,
        number_of_steps=10,
        algorithm=Algorithm(
            kisao_id='KISAO_0000019',
        ),
    ),
)
sbml_uri = 'http://www.sbml.org/sbml/level2/version4'
variables = [
    Variable(
        id='time',
        symbol=Symbol.time.value,
        task=task,
    ),    
    Variable(
        id='clb',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Clb"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
]

In [23]:
import biosimulators_copasi
import biosimulators_tellurium

In [24]:
copasi_outputs, copasi_log = biosimulators_copasi.exec_sed_task(task, variables)
tellurium_outputs, tellurium_log = biosimulators_tellurium.exec_sed_task(task, variables)

/usr/local/lib/python3.9/site-packages/kisao/utils.py:477: AlgorithmSubstitutedWarning:

'LSODA/LSODAR hybrid method' (KISAO_0000560) will be substituted for 'CVODE'' (KISAO_0000019) at substitution policy 'SIMILAR_VARIABLES'.



<div class="alert alert-block alert-info">Note, COPASI raised a warning that indicated that it used LSODA (<a href="https://www.ebi.ac.uk/ols/ontologies/kisao/terms?iri=http%3A%2F%2Fwww.biomodels.net%2Fkisao%2FKISAO%23KISAO_0000560">KISAO_0000560</a>) instead of CVODE.</div>

Next, use the logs to inspect which algorithms COPASI and tellurium used.

In [25]:
print(f'COPASI: {copasi_log.algorithm}')
print(f'tellurium: {tellurium_log.algorithm}')

COPASI: KISAO_0000560
tellurium: KISAO_0000019


## 6. Disabling optional BioSimulators features, such as to increase simulation performance

By default, BioSimulators simulation tool APIs validate models, simulations, algorithms, and simulation results; substitute unsupported algorithms with similar ones; and log the execution of simulations. Optionally, these features can be disabled, such as for more performant simulation.

These features can be disabled by creating a configuration object and passing it to `exec_sed_task` as illustrated below.
* Disable validation: set `VALIDATE_SEDML`, `VALIDATE_SEDML_MODELS`, and `VALIDATE_RESULTS` to `False`
* Disable logging: set `LOG` to `False`

<div class="alert alert-block alert-info">
    Note:
    <ul>
        <li>Without validation, simulation tools may generate cryptic exceptions.</li>
        <li>When algorithm substitution is disabled, simulation tools will raise exceptions for simulations they cannot execute.</li>
        <li>When logging is disabled, the <code>log</code> output will be <code>None</code>.</li>
    </ul>
</div>

In [26]:
from biosimulators_utils.config import Config
from kisao import AlgorithmSubstitutionPolicy
config = Config(
    VALIDATE_SEDML=False,
    VALIDATE_SEDML_MODELS=False,
    VALIDATE_RESULTS=False,
    ALGORITHM_SUBSTITUTION_POLICY=AlgorithmSubstitutionPolicy.SAME_METHOD,
    LOG=False,
)

In [27]:
outputs, log = biosimulators_tellurium.exec_sed_task(task, variables, config=config)

The following code illustrates how these options prevent COPASI from executing the simulation.

In [28]:
from kisao.exceptions import AlgorithmCannotBeSubstitutedException
try:
    biosimulators_copasi.exec_sed_task(task, variables, config=config)
except AlgorithmCannotBeSubstitutedException as exception:
    print(str(exception))

Algorithms cannot be substituted at policy 'SAME_METHOD'.


## 7. Optimizing the execution of repeated simulations by preprocessing simulation tasks

BioSimulators uses SED-ML and KiSAO to provide investigators consistent APIs for executing simulations with multiple simulation tools. This simplicity has several advantages, including eliminating the need to learn distinct APIs for each tool, making it easy to run simulations with multiple tools such as to check reproducibility and validate simulation tools, and making it easier for investigators to co-simulate models that require distinct simulation methods and simulation tools. As a result, executing simulations through SED-ML and KiSAO requires simulation tools to map descriptions of models and simulations encoded in SED-ML and KiSAO into the simulation tool's internal representations.

To avoid repeatedly calculating these maps, these maps can be calculated using the `preprocess_sed_task` method of each simulator's API and passed to `exec_sed_task` as illustrated below.

`preprocess_sed_task` enables minor changes to be efficiently made to models and simulations:
* Modify the values of parameters and initial conditions of models.
* Modify the attributes of time coure simulations (initial time, output start time, output end time, number of steps).

Note, `preprocess_sed_task` must be re-run to make more substantial changes to model and simulations:
* Additional model attributes which were not preprocessed must be changed.
* The structure of the model must be changed, or a different model must be simulated.
* A simulation must be executed with a different algorithm or algorithm parameter.
* Additional outputs which were not preprocessed must be recorded.
* A simulation must be executed with a different simulation tool.

Run the following code to preprocess the Ciliberto simulation for tellurium.

In [29]:
preprocessed_task = biosimulators_tellurium.preprocess_sed_task(task, variables, config=config)

Run the following code to pass the preprocessed information about the task to tellurium.

In [30]:
outputs, _, = biosimulators_tellurium.exec_sed_task(task, variables, preprocessed_task=preprocessed_task, config=config)

The following code illustrates how to run multiple steps with tellurium with different initial conditions.

In [31]:
task = Task(
    model=Model(
        source='../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml',
        language=ModelLanguage.SBML.value,        
        changes=[
            ModelAttributeChange(
                target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Clb"]',
                target_namespaces={'sbml': sbml_uri},
            ),
            ModelAttributeChange(
                target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Cln"]',
                target_namespaces={'sbml': sbml_uri},
            ),
            ModelAttributeChange(
                target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="SBF"]',
                target_namespaces={'sbml': sbml_uri},
            ),
            ModelAttributeChange(
                target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Sic"]',
                target_namespaces={'sbml': sbml_uri},        
            ),
        ]
    ),
    simulation=UniformTimeCourseSimulation(
        initial_time=0.,
        output_start_time=0.,
        output_end_time=10.,
        number_of_steps=10,
        algorithm=Algorithm(
            kisao_id='KISAO_0000019',
        ),
    ),
)
sbml_uri = 'http://www.sbml.org/sbml/level2/version4'
variables = [
    Variable(
        id='time',
        symbol=Symbol.time.value,
        task=task,
    ),    
    Variable(
        id='clb',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Clb"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
    Variable(
        id='cln',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Cln"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
    Variable(
        id='sbf',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="SBF"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
    Variable(
        id='sic1',
        target='/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id="Sic"]',
        target_namespaces={'sbml': sbml_uri},
        task=task,
    ),
]
preprocessed_task = biosimulators_tellurium.preprocess_sed_task(task, variables, config=config)

time_step = 1.

# set initial conditions and execute a simulation step
task.simulation.initial_time = 0.
task.simulation.output_start_time = 0.
task.simulation.output_end_time = 1.
task.model.changes[0].new_value = 0.18453673
task.model.changes[1].new_value = 0.053600963
task.model.changes[2].new_value = 0.12405464
task.model.changes[3].new_value = 0.0035491784
outputs, _, = biosimulators_tellurium.exec_sed_task(task, variables, preprocessed_task=preprocessed_task, config=config)

# set the input to the next step equal to the output of the previous step, and execute a second simulation step
task.simulation.initial_time += time_step
task.simulation.output_start_time += time_step
task.simulation.output_end_time += time_step
task.model.changes[0].new_value = outputs['clb'][-1]
task.model.changes[1].new_value = outputs['cln'][-1]
task.model.changes[2].new_value = outputs['sbf'][-1]
task.model.changes[3].new_value = outputs['sic1'][-1]
outputs, _, = biosimulators_tellurium.exec_sed_task(task, variables, preprocessed_task=preprocessed_task, config=config)

## 8. Executing simulations with commercially-licensed solvers

Some simulation tools can optionally execute simulations more quickly with commercial solvers such as [Gurobi](https://www.gurobi.com/products/gurobi-optimizer/) and [IBM CPLEX](https://www.ibm.com/analytics/cplex-optimizer). Gurobi and IBM both provide free licenses for academic research. BioSimulators recommends Gurobi because Gurobi is easier to install, Gurobi provides APIs for a wide range of versions of Python, and it is easier to manage licenses for Gurobi.

Currently, COBRApy and RBApy can be used with both CPLEX and Gurobi. To use CPLEX, you must also install CPLEX and its Python API. Note, CPLEX currently only provides Python APIs for Python 3.7 and 3.8. Gurobi can be installed by running `pip install gurobipy` or installing the BioSimulators interfaces to COBRApy and RBApy with the `gurobi` installation option (e.g., `pip install biosimulators-cobrapy[gurobi]`.

COBRApy and RBApy provide two ways to use Gurobi licenses.
- Save a copy of your license file to your home directory (`~/gurobi.lic`) or the default location (`/opt/gurobi/gurobi.lic` for Linux).
- Set environment variables with the prefix `GRB_` for each variable of your license (e.g., `GRB_WLSACCESSID`, `GRB_WLSSECRET`, `GRB_LICENSEID`).

BioSimulators recommends the former for local development and the latter for use with containers such as in cloud deployments and cloud continuous integration actions.

<div class="alert alert-block alert-info">
    Note, Gurobi provides three different licensing options for use inside and outside of containers.
</div>